In [62]:
import arcpy
from arcpy import env
import os
import numpy as np
from arcgis import GIS
from arcgis.features import GeoAccessor
from arcgis.features import GeoSeriesAccessor
import pandas as pd
# import geopandas as gpd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.pivot_table(df, values='a', index='b', columns='c', aggfunc='sum', fill_value=0)
# pd.DataFrame.spatial.from_featureclass(???)  
# df.spatial.to_featureclass(location=???,sanitize_columns=False)  

# gsa = arcgis.features.GeoSeriesAccessor(df['SHAPE'])  
# df['AREA'] = gsa.area  # KNOW YOUR UNITS

In [63]:
# fill NA values in Spatially enabled dataframes (ignores SHAPE column)
def fill_na_sedf(df_with_shape_column, fill_value=0):
    if 'SHAPE' in list(df_with_shape_column.columns):
        df = df_with_shape_column.copy()
        shape_column = df['SHAPE'].copy()
        del df['SHAPE']
        return df.fillna(fill_value).merge(shape_column,left_index=True, right_index=True, how='inner')
    else:
        raise Exception("Dataframe does not include 'SHAPE' column")

In [64]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "scratch.gdb", 'housing_transportation_2017_ACS2019.gdb']
gdb = os.path.join(outputs[0], outputs[1])
gdb2 = os.path.join(outputs[0], outputs[2])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

if not arcpy.Exists(gdb2):
    arcpy.CreateFileGDB_management(outputs[0], outputs[2])

In [65]:
ht = pd.read_csv(r".\Inputs\htaindex2019_data_blkgrps_49.csv")
block_groups = pd.DataFrame.spatial.from_featureclass(r".\Inputs\census_blocks_2019.shp")
city_area = pd.DataFrame.spatial.from_featureclass(r'.\Inputs\city_area.shp')

In [66]:
city_area

,FID,CityArea,REMM,DEVACRES,Shape_Leng,Shape_Area,SHAPE
0,0,Alpine,1,4609.200195,24217.016211,1.968129e+07,"{""rings"": [[[434465.82019999996, 4482406.7413]..."
1,1,Alta,0,985.791016,14540.509726,1.108802e+07,"{""rings"": [[[447368.2999999998, 4493854.1], [4..."
2,2,American Fork,1,8668.790039,29876.673897,3.552827e+07,"{""rings"": [[[433816.9132000003, 4473424.775800..."
3,3,Balance of BOX ELDER,0,16002.000000,142028.435611,2.007926e+08,"{""rings"": [[[411467.5, 4585959.199999999], [41..."
4,4,Benjamin,1,4309.799805,17920.195961,1.744116e+07,"{""rings"": [[[440532.13320000004, 4440745.1259]..."
...,...,...,...,...,...,...,...
104,104,Westside Mountains,1,65800.398438,334077.838665,4.943617e+08,"{""rings"": [[[402948.80009999964, 4480597.30000..."
105,105,White City,1,561.570007,6464.643755,2.549312e+06,"{""rings"": [[[426289.5, 4491709.5], [426315, 44..."
106,106,Willard,0,3871.070068,22889.218765,1.590732e+07,"{""rings"": [[[412660.7999999998, 4588752.4], [4..."
107,107,Woodland Hills,1,1541.060059,10722.729680,6.624176e+06,"{""rings"": [[[444229.13320000004, 4431005.50589..."


In [67]:
ht['blkgrp'] = ht['blkgrp'].str.replace('"', '')

# set zeros to NA, to avoid them being averaged
ht.loc[ht['t_ami'] == 0,  't_ami'] = np.nan
ht.loc[ht['h_ami'] == 0,  't_ami'] = np.nan
ht.loc[ht['ht_ami'] == 0,  't_ami'] = np.nan

In [68]:
ht_shp = ht.merge(block_groups, left_on='blkgrp', right_on='GEOID', how='left')
ht_shp = ht_shp[['blkgrp', 'h_ami', 't_ami', 'ht_ami', 'SHAPE']].copy()

In [69]:
ht_shp.spatial.to_featureclass(location=os.path.join(gdb, 'housing_transportation_costs_2017'),sanitize_columns=False) 

'e:\\Projects\\Regional-Metrics-Dashboard\\housing-plus-transportation\\Outputs\\scratch.gdb\\housing_transportation_costs_2017'

In [74]:
ht_small_area = pd.DataFrame.spatial.from_featureclass(r'.\Outputs\scratch.gdb\housing_transportation_costs_2017_by_smallarea')
ht_city = pd.DataFrame.spatial.from_featureclass(r'.\Outputs\scratch.gdb\housing_transportation_costs_2017_by_city')
lu = pd.DataFrame.spatial.from_featureclass(r'E:\Projects\Regional-Metrics-Dashboard\housing-plus-transportation\Outputs\scratch.gdb\cityarea_smallarea_sj')
lu = lu[['CityArea', 'NewSA', 'CO_NAME']].copy()

In [75]:
ht_city

,OBJECTID,CityArea,REMM,DEVACRES,Shape_Leng,mean_h_ami,mean_t_ami,mean_ht_ami,SHAPE
0,1,Alpine,1,4609.200195,24217.016211,39.428571,24.000000,63.714286,"{""rings"": [[[434465.82019999996, 4482406.7413]..."
1,2,Alta,0,985.791016,14540.509726,NaN,NaN,NaN,"{""rings"": [[[447368.2999999998, 4493854.1], [4..."
2,3,American Fork,1,8668.790039,29876.673897,24.736842,22.000000,46.578947,"{""rings"": [[[433816.9132000003, 4473424.775800..."
3,4,Balance of BOX ELDER,0,16002.000000,142028.435611,22.250000,26.750000,49.250000,"{""rings"": [[[411467.5, 4585959.199999999], [41..."
4,5,Benjamin,1,4309.799805,17920.195961,NaN,NaN,NaN,"{""rings"": [[[440532.13320000004, 4440745.1259]..."
...,...,...,...,...,...,...,...,...,...
104,105,Westside Mountains,1,65800.398438,334077.838665,NaN,NaN,NaN,"{""rings"": [[[402948.80009999964, 4480597.30000..."
105,106,White City,1,561.570007,6464.643755,23.000000,21.750000,44.500000,"{""rings"": [[[426289.5, 4491709.5], [426315, 44..."
106,107,Willard,0,3871.070068,22889.218765,NaN,NaN,NaN,"{""rings"": [[[412660.7999999998, 4588752.4], [4..."
107,108,Woodland Hills,1,1541.060059,10722.729680,NaN,NaN,NaN,"{""rings"": [[[444229.13320000004, 4431005.50589..."


In [76]:
outputs[2]

'housing_transportation_2017_ACS2019.gdb'

In [77]:
ht_city = ht_city[['CityArea', 'mean_h_ami','mean_t_ami','mean_ht_ami','SHAPE']].copy()
ht_city['mean_h_ami'] = ht_city['mean_h_ami'].round(2)
ht_city['mean_t_ami'] = ht_city['mean_t_ami'].round(2)
ht_city['mean_ht_ami'] = ht_city['mean_ht_ami'].round(2)
ht_city = ht_city.merge(lu, on ='CityArea', how='left')
ht_city.columns = ['CityArea', 'h_ami_2017','t_ami_2017','ht_ami_2017', 'SHAPE', 'SUBAREA', 'CO_NAME']
ht_city = ht_city[['CityArea', 'SUBAREA', 'CO_NAME','h_ami_2017','t_ami_2017','ht_ami_2017','SHAPE']]
# ht_city = GeoAccessor.from_df(ht_city, geometry_column='SHAPE')
ht_city.spatial.to_featureclass(location=os.path.join(gdb2, 'housing_transit_costs_2017_ACS2019'),sanitize_columns=False)  

'e:\\Projects\\Regional-Metrics-Dashboard\\housing-plus-transportation\\Outputs\\housing_transportation_2017_ACS2019.gdb\\housing_transit_costs_2017_ACS2019'

In [78]:
# ht_small_area = ht_small_area[['NewSA', 'mean_h_ami','mean_t_ami','mean_ht_ami','SHAPE']].copy()
# ht_small_area['mean_h_ami'] = ht_small_area['mean_h_ami'].round(2)
# ht_small_area['mean_t_ami'] = ht_small_area['mean_t_ami'].round(2)
# ht_small_area['mean_ht_ami'] = ht_small_area['mean_ht_ami'].round(2)
# ht_small_area.columns = ['SmallArea', 'h_ami_2017','t_ami_2017','ht_ami_2017','SHAPE']